In [7]:
!pip3 install pyspark
!pip3 install duckdb
!wget -O "HR_comma_sep.csv" "https://mydisk.cs.upc.edu/s/3o33yciBHADiFCD/download/HR_comma_sep.csv"
!wget -O "duckdb.jar" "https://repo1.maven.org/maven2/org/duckdb/duckdb_jdbc/0.10.1/duckdb_jdbc-0.10.1.jar"
import pyspark
from pyspark.sql import SparkSession
import duckdb

"""conn = duckdb.connect("exemple.db") 
conn.close()"""
spark = SparkSession.builder\
    .config("spark.jars", "duckdb.jar") \
    .getOrCreate()

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
--2024-04-10 20:12:40--  https://mydisk.cs.upc.edu/s/3o33yciBHADiFCD/download/HR_comma_sep.csv
S'està resolent mydisk.cs.upc.edu (mydisk.cs.upc.edu)… 147.83.29.248
S'està connectant a mydisk.cs.upc.edu (mydisk.cs.upc.edu)|147.83.29.248|:443… connectat.
HTTP: s'ha enviat la petició, s'està esperant una resposta… 200 OK
Mida: 566778 (553K) [text/csv]
S'està desant a: «HR_comma_sep.csv»

HR_comma_sep.csv    100%[===================>] 553,49K  --.-KB/s    en 0,1s    

2024-04-10 20:12:40 (3,87 MB/s) - s'ha desa

In [3]:
HR = spark.read.option("delimiter", ",").option("header", True).parquet("data_lake/compravenda_sup.parquet")
HR.show()

+----+---------+--------------+--------------+----------+--------------------+----------------------------------+------+
| Any|Trimestre|Codi_Districte| Nom_Districte|Codi_Barri|           Nom_Barri|Superfície_mitjana_(m2_construïts)|Nombre|
+----+---------+--------------+--------------+----------+--------------------+----------------------------------+------+
|2023|        1|             1|  Ciutat Vella|         1|            el Raval|                             Total|  63.8|
|2023|        1|             1|  Ciutat Vella|         2|      el Barri Gòtic|                             Total|  82.9|
|2023|        1|             1|  Ciutat Vella|         3|      la Barceloneta|                             Total|  41.3|
|2023|        1|             1|  Ciutat Vella|         4|Sant Pere, Santa ...|                             Total|  70.9|
|2023|        1|             2|      Eixample|         5|       el Fort Pienc|                             Total|  80.6|
|2023|        1|             2| 

In [4]:
HR.write \
    .format("jdbc") \
    .option("url", "jdbc:duckdb:exemple_compravenda.db") \
    .option("dbtable", "HR") \
    .option("driver", "org.duckdb.DuckDBDriver") \
    .save()

In [9]:
DF = spark.read \
  .format("jdbc") \
  .option("url", "jdbc:duckdb:exemple_compravenda.db") \
  .option("driver", "org.duckdb.DuckDBDriver") \
  .option("query", "SELECT salary, avg(last_evaluation::float) FROM HR GROUP BY salary") \
  .load()

DF.show()

Py4JJavaError: An error occurred while calling o67.load.
: java.sql.SQLException: java.sql.SQLException: Binder Error: Referenced column "salary" not found in FROM clause!
Candidate bindings: "HR.Any"
LINE 1: ...st_evaluation::float) FROM HR GROUP BY salary, last_evaluation) SPARK_GEN_SUBQ...
                                                  ^
	at org.duckdb.DuckDBPreparedStatement.prepare(DuckDBPreparedStatement.java:122)
	at org.duckdb.DuckDBPreparedStatement.<init>(DuckDBPreparedStatement.java:76)
	at org.duckdb.DuckDBConnection.prepareStatement(DuckDBConnection.java:76)
	at org.duckdb.DuckDBConnection.prepareStatement(DuckDBConnection.java:188)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRDD$.getQueryOutputSchema(JDBCRDD.scala:65)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRDD$.resolveTable(JDBCRDD.scala:58)
	at org.apache.spark.sql.execution.datasources.jdbc.JDBCRelation$.getSchema(JDBCRelation.scala:241)
	at org.apache.spark.sql.execution.datasources.jdbc